## **Ecobici data**

Trabajando con datos abiertos del gobierno de CABA referidos a recorridos/viajes realizados en las biciclietas del sistema de Bike Sharing de CABA, "EcoBici". Los datos comprenden el periodo 2010-2019.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

import pandas as pd
import json

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("ecobici") \
        .getOrCreate()

Los archivos no mantienen un esquema homogéneo entre sí. Tienen diferentes números y nombres de columna.

Previamente, se ha trabajado para descartar algunas columnas y obtener un archivo JSON con los nombres de las columnas para uno de los archivos.

In [3]:
with open("files_schema.json", "r") as json_file:
    columns_per_files = json.load(json_file)
columns_per_files

{'recorridos-realizados-2010.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'long_estacion_origen',
  'lat_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  'id_estacion_destino',
  'long_estacion_destino',
  'lat_estacion_destino'],
 'recorridos-realizados-2011.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'long_estacion_origen',
  'lat_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  'id_estacion_destino',
  'long_estacion_destino',
  'lat_estacion_destino'],
 'recorridos-realizados-2012.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'lat_estacion_origen',
  'long_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  'id_estacion_destino',
  'lat_estacion_destino',
  'long_estacion_destino'],
 'recorridos-realizados-2013.csv': ['fecha_origen_recorrido',
  'id_estacion_origen',
  'long_estacion_origen',
  'lat_estacion_origen',
  'duracion_recorrido',
  'fecha_destino_recorrido',
  '

### **Null values**
Primero contaremos los valores nulos para cada una de las columnas de cada archivo

In [4]:
for file, columns in columns_per_files.items():
    print(file)
    rides = spark.read.csv(file, header=True)
    rides = rides.select(*columns)
    for column in columns:
        print(f"Column: {column}\t Nulls: {rides.filter(F.isnull(column)).count()}")

recorridos-realizados-2010.csv
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 0
Column: long_estacion_origen	 Nulls: 0
Column: lat_estacion_origen	 Nulls: 0
Column: duracion_recorrido	 Nulls: 3
Column: fecha_destino_recorrido	 Nulls: 0
Column: id_estacion_destino	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
recorridos-realizados-2011.csv
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 85
Column: long_estacion_origen	 Nulls: 85
Column: lat_estacion_origen	 Nulls: 85
Column: duracion_recorrido	 Nulls: 148
Column: fecha_destino_recorrido	 Nulls: 0
Column: id_estacion_destino	 Nulls: 178
Column: long_estacion_destino	 Nulls: 178
Column: lat_estacion_destino	 Nulls: 178
recorridos-realizados-2012.csv
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 2487
Column: lat_estacion_origen	 Nulls: 2487
Column: long_estacion_origen	 Nulls: 2487
Column: duracion_recorrido	 Nu

El análisis de estos valores nulos se puede separar de acuerdo a grupos de columnas.

Con respecto a los campos ***id_estacion***, ***lat*** y ***long*** (tanto para la estación origen como destino): Mantienen el mismo número de valores nulos entre sí, podemos borrar aquellas filas con valores nulos para el campo id_estacion (ya sea origen o destino) ya que sin ese id no es posible obtener otros datos de la estacion.

Sobre las ***fechas*** y ***duración*** del recorrido: Todos los archivos tienen una fecha de origen del recorrido. Si hay filas con valores nulos para los campos duracion y fecha de destino del recorrido, se descartan. Si solo una de esas fuese nula, puede obtener mediante funciones que correspondan.

### **Deleting null values**

In [5]:
for file, columns in columns_per_files.items():
    print(file)
    rides = spark.read.csv(file, header=True)
    rides = rides.select(*columns)
    
    # Deleting rows about origin station
    cols = columns[1:4]
    print(f"Columns to inspect: {cols}")
    rides = rides.dropna(how="all", subset=cols)
    
    # Deleting rows about target station
    cols = columns[6:]
    print(f"Columns to inspect: {cols}")
    rides = rides.dropna(how="all", subset=cols)
    
    # Deleting rows about date and duration ride
    cols = columns[4:6]
    print(f"Columns to inspect: {cols}")
    rides = rides.dropna(how="all", subset=cols)
    
    for column in columns:
        print(f"Column: {column}\t Nulls: {rides.filter(F.isnull(column)).count()}")
    
    # Writing each dataframe a in unified table
    #tablename = f"rides-{file[-8:-4]}"
    tablename = f"rides{file[-8:-4]}" # rides string + year
    rides.write.saveAsTable(tablename)

recorridos-realizados-2010.csv
Columns to inspect: ['id_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen']
Columns to inspect: ['id_estacion_destino', 'long_estacion_destino', 'lat_estacion_destino']
Columns to inspect: ['duracion_recorrido', 'fecha_destino_recorrido']
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 0
Column: long_estacion_origen	 Nulls: 0
Column: lat_estacion_origen	 Nulls: 0
Column: duracion_recorrido	 Nulls: 3
Column: fecha_destino_recorrido	 Nulls: 0
Column: id_estacion_destino	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
recorridos-realizados-2011.csv
Columns to inspect: ['id_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen']
Columns to inspect: ['id_estacion_destino', 'long_estacion_destino', 'lat_estacion_destino']
Columns to inspect: ['duracion_recorrido', 'fecha_destino_recorrido']
Column: fecha_origen_recorrido	 Nulls: 0
Column: id_estacion_origen	 Nulls: 0
Col

### **Filling nulls values**

In [6]:
def change_column_dtype(df, column, new_dtype):
    df = df.withColumn(column, \
                             df[column] \
                              .cast(new_dtype) \
                              .alias(column))
    return df

Se cuenta con un dataset con datos como id, nombre, latitud, longitud, direccion, etc. de cada estacion de ecobici.

In [7]:
stations = spark.read.csv("estaciones-bicicletas-publicas.csv", header=True)
stations = stations.repartition(1)
# stations.rdd.getNumPartitions()
print(*stations.columns)

lat long nombre_estacion id_estacion capacidad dirección_completa direccion_nombre direccion_altura direccion_interseccion barrio


In [8]:
stations = change_column_dtype(stations, "id_estacion", "int")
stations.select("id_estacion").dtypes

[('id_estacion', 'int')]

Primero trabajaremos con los campos referidos a **latitud** y **longitud**, donde solo los archivos de los años 2015 y 2019 presentaban valores nulos en dichos campos.

In [9]:
for year in ["2015", "2019"]:
    print(f"rides{year}")
    rides = spark.table(f"rides{year}")
    rides.filter(F.isnull("lat_estacion_origen") & \
                F.isnull("long_estacion_origen")) \
        .select("id_estacion_origen").distinct().show()
    
    rides.filter(F.isnull("lat_estacion_destino") & \
                F.isnull("long_estacion_destino")) \
        .select("id_estacion_destino").distinct().show()

rides2015
+------------------+
|id_estacion_origen|
+------------------+
|                15|
|                39|
+------------------+

+-------------------+
|id_estacion_destino|
+-------------------+
|                 15|
|                 39|
+-------------------+

rides2019
+------------------+
|id_estacion_origen|
+------------------+
|             159_0|
|              44_0|
+------------------+

+-------------------+
|id_estacion_destino|
+-------------------+
|              159_0|
|               44_0|
+-------------------+



Respecto a los particulares ids "159_0" y "44_0", los verificaremos con el dataset de estaciones

In [10]:
stations.filter((stations.id_estacion == 15) | (stations.id_estacion == 39) | \
               (stations.id_estacion == 159) | (stations.id_estacion == 44) | \
               (stations.id_estacion == "44_0") | (stations.id_estacion == "159_0")) \
        .select("id_estacion").show()

+-----------+
|id_estacion|
+-----------+
|         15|
|         39|
|         44|
+-----------+



La estacion de id "159_0" o 159 no existe, se puede descartar las filas con ese valor.
Luego, nos aseguramos que el valor de id "44_0", corresponde a 44.

In [12]:
spark.read.csv("recorridos-realizados-2019.csv", header=True) \
.filter("id_estacion_origen = '44_0'") \
.select("nombre_estacion_origen").show(1)

stations.filter("id_estacion == 44").select("nombre_estacion").show()

+----------------------+
|nombre_estacion_origen|
+----------------------+
|             Ecoparque|
+----------------------+
only showing top 1 row

+---------------+
|nombre_estacion|
+---------------+
|044 - Ecoparque|
+---------------+



Efectivamente, coinciden. Ahora continuamos con los tratamientos correspondientes

In [14]:
rides = spark.table("rides2019")
rides = rides.replace("44_0", "44", subset=["id_estacion_origen", "id_estacion_destino"])
rides = change_column_dtype(rides, "id_estacion_origen", "int")
rides = change_column_dtype(rides, "id_estacion_destino", "int")
rides = rides.na.drop(how="any", subset=["id_estacion_origen", "id_estacion_destino"])
rides.write.saveAsTable("temp")
spark.table("temp").write.insertInto("rides2019", overwrite=True)

Finalmente, ya podemos completar los valores nulos para los campos de latidad y longitud

In [16]:
def filling_geographical_fields(rides_df, stations_df, coord_type, station_type):
    """
    coord_type: lat/long
    station_type: origen/destino
    """
    stations_df = stations_df.select(*["id_estacion", "lat", "long"])
    stations_df = stations_df.withColumn(f"id_estacion_{station_type}", \
                                        F.col("id_estacion"))
    
    joined_df = rides_df.join(stations_df, on=f"id_estacion_{station_type}")
    joined_df = joined_df.withColumn(f"{coord_type}_estacion_{station_type}", \
                                F.coalesce(joined_df[f"{coord_type}_estacion_{station_type}"], \
                                          joined_df[coord_type]))
    joined_df = joined_df.drop("id_estacion", "lat", "long")
    return joined_df

In [23]:
for year in ["2015", "2019"]:
    rides = spark.table(f"rides{year}")
    for station_type in ["origen", "destino"]:
        for coord_type in ["lat", "long"]:
            rides = filling_geographical_fields(rides, stations, coord_type, station_type)
            
    print(f"rides{year}")
    for column in rides.columns:
        if 'lat' in column or 'long' in column:
            print(f"Column: {column}\t Nulls: {rides.filter(F.isnull(column)).count()}")
    
    rides.write.saveAsTable("temp", mode="overwrite")
    spark.table("temp").write.insertInto(f"rides{year}", overwrite=True)
        
    

rides2015
Column: long_estacion_origen	 Nulls: 0
Column: lat_estacion_origen	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
rides2019
Column: lat_estacion_origen	 Nulls: 0
Column: long_estacion_origen	 Nulls: 0
Column: lat_estacion_destino	 Nulls: 0
Column: long_estacion_destino	 Nulls: 0
